# Assessment Problems Notebook

## Problem 1 - Extending the Lady Tasting Tea

In this notebook, I will extend the Lady Tasting Tea experiment.

- Original Fisher design: 8 cups total (4 tea-first, 4 milk-first).
- Extended design: 12 cups total (8 tea-first, 4 milk-first).

A participant claims they can tell, for each cup, whether the milk or tea was poured first.  
We want to estimate, by simulation, the probability that a participant with no real ability (i.e. just guessing) correctly identifies all cups in each design.

I will:

1. Simulate the experiment many times using numpy.
2. Estimate the probability of perfectly identifying all cups by chance in:
   - The original 8-cup experiment.
   - The extended 12-cup experiment.
3. Compare the estimated probabilities and discuss what this means for p-values and evidence against the null hypothesis.

In [1]:
# Import Libraries

import math
import itertools
import numpy as np

## Simulation

I randomly assign 4 cups as milk‑first and randomly guess 4 cups.


If the person is just guessing, both choices are random. The proportion of trials where the two sets match is an estimate of the probability of getting all cups correct by chance.

I have decided to run this using 50,000 random trials

In [5]:
# Simulation function
def simulate(n_trials=50000):
    cups = np.arange(12)
    count = 0
    for _ in range(n_trials):
        true_milk = set(np.random.choice(cups,4,replace=False))
        guess = set(np.random.choice(cups,4,replace=False))
        if true_milk == guess:
            count += 1
    return count/n_trials

sim_prob = simulate()
sim_prob

0.00176

Now to compare the 12 cup test with the 8 cup test

In [7]:
exact_prob = 1 / math.comb(12,4)
orig_prob = 1 / math.comb(8,4)
exact_prob, orig_prob

(0.00202020202020202, 0.014285714285714285)

### Interpretation

The 12 cup test makes it much rarer to get a correct guess making this test stronger compared to the 8 cup test.

# Problem 2 - Normal Distribution